In [1]:
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import utils

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
from IPython.display import Image

import os
import h5py

In [2]:
path = os.path.join(os.getcwd(), "dataset/input/")

hf = h5py.File(path + "train_snoring_5s.h5", 'r')

x_train = np.array(hf.get('spectrograms'))
y_train = np.array(hf.get('targets')).astype(np.int8)
hf.close()
print(x_train.shape)
print(y_train.shape)

hf = h5py.File(path + "test_snoring_5s.h5", 'r')

x_test = np.array(hf.get('spectrograms'))
y_test = np.array(hf.get('targets')).astype(np.int8)
hf.close()
print(x_test.shape)
print(y_test.shape)

(19008, 129, 155)
(19008,)
(4752, 129, 155)
(4752,)


In [3]:
print(x_train.dtype)
print(y_train.dtype)
print(x_test.dtype)
print(y_test.dtype)

float64
int8
float64
int8


In [4]:
print(x_train[0])
print(y_train[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 2. ... 1. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0


In [6]:
## parameter setting
bands = 129
frames = 155
feature_size = 3000
learning_rate = 0.02
training_epochs = 100
batch_size = 5
num_of_class = 2
channel=1

In [7]:
x_train = np.reshape(x_train,(-1,bands,frames,1))
x_test = np.reshape(x_test,(-1,bands,frames,1))
# If subtract pixel mean is enabled

y_train = utils.to_categorical(y_train, num_of_class)
y_test = utils.to_categorical(y_test, num_of_class)

In [8]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(19008, 129, 155, 1)
(19008, 2)
(4752, 129, 155, 1)
(4752, 2)


In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Concatenate, Input, BatchNormalization, ELU
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [19]:
# intput
inp = Input(shape=(bands, frames, channel), name='inp')

# act = keras.layers.advanced_activations.LeakyReLU(alpha=0.3)
act = ELU(alpha=1.0)


f = Conv2D(16, (10, 10), strides=1, padding='same', activation=act, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform', input_shape=(bands, frames, channel))(inp)
#f = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(f)

f = Conv2D(16, (10, 10), strides=1, padding='same',activation=act, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')(f)
#f = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(f)
#f = MaxPooling2D(pool_size=(2, 2), padding='same')(f)
f = Conv2D(16, (10, 10), padding='same',activation=act, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')(f)
f = Conv2D(16, (10, 10), strides=1, padding='same',activation=act, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')(f)
#f = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(f)
f = MaxPooling2D(pool_size=(2, 2), padding='same')(f)
f = Conv2D(16, (10, 10), padding='same',activation=act, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')(f)
f = Conv2D(16, (10, 10), strides=1, padding='same',activation=act, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')(f)
#f = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(f)

f = Conv2D(16, (10, 10), padding='same',activation=act, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')(f)
#f = MaxPooling2D(pool_size=(2, 2), padding='same')(f)
f = Conv2D(16, (10, 10), strides=1, padding='same',activation=act, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')(f)
#f = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(f)

f = Conv2D(16, (10, 10), padding='same',activation=act, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')(f)

f = MaxPooling2D(pool_size=(2, 2), padding='same')(f)

print(f)
outf = Flatten()(f)
print(outf)

modelf = Model(inp, outf)

out = Dense(512, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform', activation='sigmoid')(modelf.output)
out = Dropout(0.3)(out)
out = Dense(num_of_class, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform', activation='softmax')(out)

modelall = Model(modelf.input, out)
print(modelall.output_shape)

KerasTensor(type_spec=TensorSpec(shape=(None, 33, 39, 16), dtype=tf.float32, name=None), name='max_pooling2d_3/MaxPool:0', description="created by layer 'max_pooling2d_3'")
KerasTensor(type_spec=TensorSpec(shape=(None, 20592), dtype=tf.float32, name=None), name='flatten_1/Reshape:0', description="created by layer 'flatten_1'")
(None, 2)


In [20]:
modelall.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inp (InputLayer)            [(None, 129, 155, 1)]     0         
                                                                 
 conv2d_9 (Conv2D)           (None, 129, 155, 16)      1616      
                                                                 
 conv2d_10 (Conv2D)          (None, 129, 155, 16)      25616     
                                                                 
 conv2d_11 (Conv2D)          (None, 129, 155, 16)      25616     
                                                                 
 conv2d_12 (Conv2D)          (None, 129, 155, 16)      25616     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 65, 78, 16)       0         
 2D)                                                             
                                                           

In [21]:
modelall.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ["accuracy"])

In [22]:
# hist = modelall.fit(x_train, y_train, epochs=training_epochs, validation_data=(x_test, y_test), verbose=0)
hist = modelall.fit(x_train, y_train, batch_size=batch_size, epochs=training_epochs, validation_data=(x_test, y_test), verbose=2)

Epoch 1/100


2022-01-14 02:38:35.307737: E tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded runtime CuDNN library: 8.0.4 but source was compiled with: 8.1.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2022-01-14 02:38:35.309592: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : UNKNOWN: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model_3/conv2d_9/Conv2D
 (defined at /home/vips/anaconda3/envs/gh_tf38/lib/python3.8/site-packages/keras/layers/convolutional.py:231)
]] [Op:__inference_train_function_3141]

Errors may have originated from an input operation.
Input Source operations connected to node model_3/conv2d_9/Conv2D:
In[0] IteratorGetNext (defined at /home/vips/anaconda3/envs/gh_tf38/lib/python3.8/site-packages/keras/engine/training.py:866)	
In[1] model_3/conv2d_9/Conv2D/ReadVariableOp:

Operation defined at: (most recent call last)
>>>   File "/home/vips/anaconda3/envs/gh_tf38/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/home/vips/anaconda3/envs/gh_tf38/lib/python3.8/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/hom

In [ ]:
plt.plot(hist.history['accuracy'], label='accuracy')
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend(loc='upper left')
plt.show()

In [ ]:
res = modelall.predict(x_test[0:1])
print(res)

plt.bar(range(10), res[0], color='red')
plt.bar(np.array(range(10)) + 0.35, y_test[0])
plt.show()

In [ ]:
loss, acc = modelall.evaluate(x_test, y_test, verbose=2)
print(loss, acc)

## 모델 저장 및 로드

In [ ]:
modelall.save("./G_CNN.h5")    # 모델과 weights를 전부 저장

In [ ]:
new_model = tf.keras.models.load_model('./G_CNN.h5')

In [ ]:
res = new_model.predict( x_test[3:4] ) 
print(res.shape)
print(res[0])
plt.bar(range(10), res[0], color='red')
plt.bar(np.array(range(10)) + 0.35, y_test[3])
plt.show()

In [ ]:
loss, acc = new_model.evaluate(x_test, y_test, verbose=2)
print(loss, acc)